In [1]:
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.feature_selection import SelectPercentile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from subprocess import check_output


In [3]:
df = pd.read_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/CNTYVARS.csv')
#df = pd.read_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\CNTYVARS.csv')
df['HSGRAD_PRCNT'] = df['HSGRAD_CY']/df['TOTPOP_CY']

In [4]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print(dtype_df[dtype_df['Column Type'] == 'object'])

         Count Column Type
783   TSEGCODE      object
784   TSEGNAME      object
854  TLIFECODE      object
855  TLIFENAME      object


In [5]:
dp = ['OBJECTID','Customers', 'UNITS', 'SALES', 'LOG_UNITS', 'TSEGCODE','TSEGNAME','TLIFECODE','TLIFENAME']

X = df.drop(dp, axis=1)

#X2 = transformer.fit_transform(X, y)

#X2.head()

In [6]:
y = df[['LOG_UNITS']]
y.shape

(3109, 1)

In [33]:
selector = SelectPercentile(f_regression, percentile=5)
columns = X.columns
selector.fit_transform(X, np.array(y))
labels = [columns[x] for x in selector.get_support(indices=True) if x]
XNew = pd.DataFrame(selector.fit_transform(X, np.array(y)), columns = labels)
columns = XNew.columns


lst = list()
for n, s in zip(columns, selector.scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['Score','Name'])
    


/usr/lib/python3/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib/python3/dist-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [34]:
inds = list()
for val in columns:
    #print(val)
    inds.append((val, XNew.columns.get_loc(val)))
    
indices = pd.DataFrame(inds, columns=['Name','XNew_Index'])
    


In [35]:
scores.sort_values(by='Score',inplace=True, ascending=True)
scores.reset_index(inplace=True)
#scores.to_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\varscores.csv')
#scores.to_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/varscores.csv')

In [36]:
scores.drop('index', inplace=True, axis = 1)

df = pd.merge(scores, indices, how='inner',left_on='Name',right_on='Name')



In [37]:
df.head()

,Score,Name,XNew_Index
0,83.414305,GED_CY,4
1,139.203632,A65I50_CY,8
2,163.368872,A65I25_CY,6
3,171.303891,A65DI35_CY,15
4,178.448690,A65DI25_CY,14


In [38]:


def calculate_vif(X, df, thresh=5.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            # Loop repeatedly until we find that all columns within our dataset
            # have a VIF value we're happy with.
            variables = df['Name']
            dropped=False
            vif = []
            new_vif = 0
            for var in variables:
                new_vif = variance_inflation_factor(X.as_matrix(), X.columns.get_loc(var))
                vif.append(new_vif)
                if np.isinf(new_vif):
                    break
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                #print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                df = df[df['Name'] != str(X.columns[maxloc])]
                dropped=True
        return X
    
finaldf = calculate_vif(XNew, df)

/home/pirwin/.local/lib/python3.5/site-packages/statsmodels/stats/outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


KeyError: 'A65I25_CY'

In [112]:
finaldf = pd.merge(df, vifdf, how='inner',left_on='Name',right_on='Name')

finaldf.tail()

,Score,Name,XNew_Index,VIF
53,893.118410,A75NW150CY,30,3302.270798
54,902.033813,A75NW100CY,29,1145.605065
55,919.153759,N05_BUS,52,5467.064322
56,926.061314,N04_BUS,51,33.369670
57,1064.765726,A75I15_CY,10,1386.569936
